Text-based Blackjack game using Python.

Play against an automated dealer, with the ability to:

* Place a bet to start the game
* Double the initial bet after hand has been dealt
* **Hit** (deal another card) or **Stand** during player's turn
* Split cards at the start of turn if both cards have the same face value
* Continue playing after a game if you still have money!

To begin, click **Run** on the toolbar above, or press **Shift+Enter** on your keyboard twice.

In [ ]:
from IPython.display import clear_output
import random

suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven',
         'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6,
          'Seven':7, 'Eight':8, 'Nine':9, 'Ten':10, 'Jack':10,
          'Queen':10, 'King':10, 'Ace':11}

playing = True
table = {'player':None,
         'dealer':None}

class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        
    def __str__(self):
        return '{} of {}'.format(self.rank, self.suit)
    
class Deck:
    def __init__(self, multiple=1):
        # on init, build deck of cards using 'multiple' number of 52-card decks
        self.deck = []  # starting with an empty list
        
        for suit in suits:
            for rank in ranks:
                self.deck.extend([Card(suit, rank) for i in range(multiple)])  # populating deck with Card objects
    
    def shuffle(self):
        random.shuffle(self.deck)
        
    def deal(self):
        return self.deck.pop()
        
    def __str__(self):
        deck_order = ''
        for card in self.deck:
            deck_order += card.__str__() + '\n'
        return deck_order
    
class Hand:
    def __init__(self):
        # on init, create an empty hand
        self.hand = []
        self.value = 0  # default value of hand
        self.aces = 0  # keep track of aces
        
        # booleans to keep track of play
        self.show_card = True
        self.is_split = False
        self.can_split = False
        self.played = False
        
        # container for splitting hand (holds another Hand instance)
        self.split = None
    
    # method to add Card object to hand
    def add_card(self, card):
        
        self.hand.append(card)
        
        # check for the following conditions:
        #   1. current hand is not already a split hand
        #   2. current hand value is >0, meaning a card exists in hand
        #   3. hand has not been played
        #   4. card being dealt has the same rank as existing card in hand.
        # 
        # if all the above are True, the hand can be split
        if not self.is_split and self.value > 0 and not self.played and (self.value == values[card.rank]):
            self.can_split = True
        
        # adjust hand value
        self.value += values[card.rank]
        
        # adjust number of Aces
        if card.rank == 'Ace':
            self.aces += 1
        
        self.ace_adjust()
        
    def ace_adjust(self):
        # when hand value is above 21 (normally bust)
        # and hand holds ace(s), adjust the value of
        # ace(s) from 11 to 1 and the hand value accordingly
        if self.value > 21 and self.aces > 0:
            adjust_value = 0
            
            for card in self.hand:
                if card.rank == 'Ace':
                    adjust_value += 1
                else:
                    adjust_value += values[card.rank]
            
            self.value = adjust_value
            
    def update(self):
        # update Hand for use in e.g. after splitting
        update_value = 0
        update_aces = 0
        
        for card in self.hand:
            update_value += values[card.rank]
            
            if card.rank == 'Ace':
                update_aces += 1
        
        self.value = update_value
        self.aces = update_aces
        
    def __str__(self):
        ret_hand = ''
        for card in self.hand:
            ret_hand += card.__str__() + '\n'
        return ret_hand            
            
class Chips:
    def __init__(self, total=100):
        self.total = total
        self.bet = 0
        self.split_bet = 0
        
        self.bet_doubled = False
        
    def win_bet(self, bet):
        self.total += bet
        # reset current bet
        self.bet = 0
        
    def lose_bet(self, bet):
        self.total -= bet
        # reset current bet
        self.bet = 0
        
def take_bet(chips):
    global playing
    
    while True:
        try:
            bet = int(input('\nYou currently have ${}.\nPlace your bet: $'.format(chips.total)))
        except:
            print('Invalid bet. Try again: $')
            continue
        else:
            # check if bet is valid
            if 0 < bet <= chips.total:
                chips.bet = bet
                break
            elif bet == 0:
                playing = False
                break
            else:  # bet amount is more than chips total
                print("You don't have ${} to bet!".format(bet))
                continue
                
def double_bet(chips, hand):
    # offer player the option to double his initial bet in return for standing after 1 hit
    while True:
        try:
            action = str(input('Would you like to Double your bet of ${}? [Y/N] : '.format(chips.bet))).upper()
        except:
            print('Unrecognized input. Try again!')
            continue
        else:
            if action not in ('YN'):
                print('Unrecognized input. Try again!')
                continue
            
            elif action == 'Y':
                chips.bet *= 2
                chips.bet_doubled = True
                hand.can_split = False
                print('You have doubled your bet! Current bet is ${}.'.format(chips.bet))
            
            break
                
def payout(chips, bet='bet', multiple=1):
    payout = getattr(chips, bet) * multiple
    chips.total += payout
    if multiple > 0:
        print('\nYou have won ${}! You now have ${}.'.format(payout, chips.total))
    else:
        print('\nYou have lost ${}! You now have ${}.'.format(abs(payout), chips.total))
    setattr(chips, bet, 0)
                
def display_hand(hand):
    print('{}Value of hand: {}'.format(hand.__str__(), hand.value))
    
def blackjack(hand):
    card_one = hand.hand[0].rank
    card_two = hand.hand[1].rank
    return (values[card_one] == 10 and card_two == 'Ace') or (card_one == 'Ace' and values[card_two] == 10)
    
def settle_score(chips):
    '''
    Function called at end of round to settle bets.
    Compares values of hands with standing bets to determine payout
    '''
    player = table['player']
    dealer = table['dealer']
    bet = 'bet'
    
    # while loop runs until not player.split 
    # exit on player.split.split (doesn't exist)
    while getattr(chips, bet) > 0:
        # dealer busts, pay bet unless player also busts
        if dealer.value > 21:    
            
            # player is not bust
            if player.value <= 21:
                payout(chips, bet)

            # tie scenario, bet is returned
            if player.value > 21:
                print('You have also busted.')
                payout(chips, bet, 0)

        # dealer doesn't bust
        else:

            # player has higher hand value (including 21), player wins
            if dealer.value < player.value <= 21:
                payout(chips, bet)

            # player has lower hand value, player loses
            if player.value < dealer.value or player.value > 21:
                payout(chips, bet, -1)
                
            # same hand value, tie scenario
            if player.value == dealer.value:
                print('You have tied with the dealer.')
                payout(chips, bet, 0)

        # if player.split, run while loop again on split hand
        if player.split:
            player = player.split
            bet = 'split_bet'
            continue            
                                              
def hit_stand(deck, hand):
    # proceed if hand has less than 5 cards and < 21
    while len(hand.hand) < 5 and hand.value < 21:
        
        # build appropriate input text and valid actions
        action_text = '\n[H]it | [S]tand '
        actions = set('HS')
        
        # hand can be split, and has not been split
        if hand.can_split:
            action_text += '| S[P]lit '
            actions.add('P')
        
        # offer to double player's bet ONLY IF:
        #   1. Bet has not been doubled
        #   2. Hand has not been played (ie. player's first move)
        #   3. Player has sufficient funds to double bet
        #   4. Player has not split starting hand
        if not chips.bet_doubled and not hand.played and (2 * chips.bet <= chips.total) and not hand.split and not hand.is_split:
            action_text += '| [D]ouble bet '
            actions.add('D')
            
        action_text += ': '

        try:
            # handle player turn
            if hand == table['player'] or hand == table['player'].split:
                
                # handle hitting after doubling down
                if chips.bet_doubled:
                    action = 'H'
                
                else:
                    action = str(input(action_text)).upper()
                    
            # handle dealer turn
            else:
                dealer = hand  # for better code legibility within this check
                player = table['player']
                
                action = 'S'

                # handling split hand breaks
                if player.split:
                    if player.split.value <= dealer.value < 22 and player.value <= dealer.value < 22:
                        # if both split hand and hand have lower values than dealer's opening value, break
                        break
                        
                    elif dealer.value <= 17:
                        action = 'H'

                # if dealer opening value is higher than player current hand value, break
                elif player.value <= dealer.value < 22:
                    break
                
                # dealer will hit when at or under soft 17
                elif dealer.value <= 17:
                    action = 'H'
                
        except:
            print('Invalid choice. Try again.')
            continue
        else:
            # validate input
            if action not in actions:
                print('ERROR: Invalid action. Please try again.')
                continue
                
            else:
                # Hit - deal a card to the current hand
                if action == 'H':
                    hand.played = True
                    
                    card = deck.deal()
                    hand.add_card(card)
                    print('Dealt {}.\n'.format(card))
                    
                    hand.ace_adjust()
                    hand.can_split = False
                    
                    display_hand(hand)
                    
                    if hand.value > 21:
                        print('>>> BUST! <<<\n')
                    elif hand.value == 21:
                        print('>>>  21!  <<<\n')
                        
                    if len(hand.hand) == 5:
                        print('End with 5 cards.')
                        
                        # automatic payout with 5 cards < 21
                        # if 5-cards 21, double payout
                        if (hand == table['player'] or hand == table['player'].split) and hand.value <= 21:
                            multiple = 2 if hand.value == 21 else 1
                            bet = 'bet' if hand == table['player'] else 'split_bet'
                            payout(chips, bet, multiple)
                            
                        break
                    
                    # Double down handler
                    if hand == table['player'] and chips.bet_doubled:
                        print('DOUBLED DOWN: Stand after one Hit.')
                        break
                                                   
                    continue
                
                # Split the current hand
                elif action == 'P' and hand.can_split:
                    # check if enough chips to split
                    if chips.bet * 2 <= chips.total:
                        chips.split_bet = chips.bet
                    
                        # split hand by creating new hand object in hand.split
                        print('Splitting {}\'s...\n'.format(hand.hand[0].rank))
                        hand.split = Hand()
                        hand.split.is_split = True

                        # split one card from hand to split-hand
                        hand.split.add_card(hand.hand.pop())
                        hand.update()                        

                        # deal one card each to hand and split-hand
                        hand.add_card(deck.deal())
                        hand.split.add_card(deck.deal())
                        
                    else:
                        # not enough chips to split hand
                        print('Not enough chips ({}) to split.'.format(chips.bet))
                        print('You have {} chips.'.format(chips.total))
                        
                    hand.can_split = False
                    display_hand(hand)
                    continue
                    
                # Double bet
                elif action == 'D':
                    double_bet(chips, hand)
                    continue
                
                # Stand - do nothing with the current hand
                else:
                    print('You have chosen to Stand.')
                    break

#########################
# GAME CODE BEGINS HERE #
#########################
# only one instance of Chips should be in existence
chips = Chips()

# Game round begins
while playing:
    # print('Creating deck...')
    deck = Deck()
    # print('Shuffling deck...')
    deck.shuffle()
    
    # create Hand instances
    for _, players in enumerate(table):
        # print('Creating hand for {}...'.format(players))
        table[players] = Hand()
        
        # toggle boolean for dealer to hide first card dealt
        if players == 'dealer':
            table[players].show_card = False

    # take player bet
    take_bet(chips)
    print('')

    if playing:
        # dealing Cards to Hands
        # print('\nDealing cards...')
        # print('Burning top card...')
        deck.deal()

        # 2 rounds of cards are sequentially dealt
        for deal_round in range(2):
            # to each player at the table
            for i, players in enumerate(table):
                card = deck.deal()
                table[players].add_card(card)

                if not table[players].show_card:
                    card = 'Hidden Card'
                    # toggle boolean to be True to show second card
                    table[players].show_card = True

                print('Dealt {} to {}.'.format(card, players))
            print('')
        # print('Cards dealt.')

        # play game
        # handle turn order
        for _, players in enumerate(table):

            hand = table[players]
            bet = 'bet'

            # if all bets have been settled (e.g. player blackjack), break
            while chips.bet > 0 or chips.split_bet > 0:

                display_turn = '| Current Turn: {} |'.format(players.capitalize())
                print('\n{:-^{width}}\n{}\n{:-^{width}}'.format('', display_turn, '', width=len(display_turn)))            

                display_hand(hand)

                if not blackjack(hand):

                    hit_stand(deck, hand)

                    # if player split his hand, play another round with the split hand
                    if hand.split:
                        hand = hand.split
                        bet = 'split_bet'
                        continue

                # handle blackjack on deal
                else:
                    if players != 'dealer':  # to account for future player expansion
                        print('>>> BLACKJACK! <<<')
                        payout(chips, bet, 2)
                        break
                    else:
                        print('>>> Dealer 21! <<<')  # dealer doesn't score blackjack
                break

        # settle bets if they have not already been settled
        settle_score(chips)
    
    # if player is broke, terminate game
    if chips.total == 0:
        print('You don\'t have any more money. Thanks for playing!')
        playing = False
        break
        
    # if not, ask if they want to play again
    while True:
        try:
            replay = str(input('Would you like to play another round? Y/N : ')).upper()
        except:
            print('Unrecognized. Try again.')
            continue
        else:
            if replay not in ('YN'):
                print('Unrecognized input. Try again.')
                continue
            elif replay == 'N':
                print('Thanks for playing!')
                playing = False
                break
            else:
                chips.bet_doubled = False
                clear_output()
                break
